Image registration to create a pixel map between top camera (A) and quadrant camera (B) with overlapping fields of view (FOV), where quadtant camera's FOV is a subset and a rotated version of top camera's FOV. 
1. Exrtact and Load Images: Extract random matching frames from the video frames for both Camera A and Camera B and load them. Use multiple images to get moe accurate map.
2. Detect Features: Use ORB (Oriented FAST and Rotated BRIEF) to detect keypoints and get descriptors. 
3. Match Features: Use a matcher (rute-Force matcher with Hamming distance) to find corresponding keypoints between the images. Keep best matches.
4. Compute Homography: Estimate the homography matrix (matrix that best transforms the matched keypoints from Camera B's frame of reference to Camera A's frame of reference) to align the images (rotation, translation, scaling).
5. Warp Images: Apply the homography to transform Camera B's images to align with Camera A's perspective.
6. Pixel map: Get the pixel map from A to B and vice versa based on this.
7. Multi-frame averaging: Perform these computation on many frames and average over individual homography matrices for more accurate results.
6. Visualise results.


This method seems to work better then other tried, becasue:

- can deal with low number of unique features in the image (feature matching algorithms failed)
- can deal with orientation and perspective differences (template matching (= find B within A) failed)

In [ ]:
"""Notebook settings and imports"""

%load_ext autoreload
%autoreload 2

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
"""File paths (example: CameraNorth and CameraTop)"""
camera_a = 'CameraTop'
camera_b = 'CameraNorth'

# Define file paths
base_path = '/ceph/aeon/aeon'
video_path = base_path + '/data/raw/AEON3/social0.2/2024-01-31T11-28-39'
video_path_a = video_path + f'/{camera_a}/{camera_a}_2024-01-31T11-00-00.avi'
video_path_b = video_path + f'/{camera_b}/{camera_b}_2024-01-31T11-00-00.avi'

# Define image output directory
output_dir = base_path + '/code/scratchpad/Orsi/pixel_mapping/example_frames'
os.makedirs(output_dir, exist_ok=True)

# 1. Load frames

In [ ]:
"""Extract some frames from videos and save them as images"""

number_of_frames = 20

# Function to get total number of frames in a video
def get_total_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return total_frames

# Function to extract and save frames
def extract_and_save_frames(video_path, frame_indices, prefix):
    cap = cv2.VideoCapture(video_path)
    for i, frame_idx in enumerate(frame_indices):
        frame_filename = os.path.join(output_dir, f'{prefix}_frame_{i}.png')
        if not os.path.exists(frame_filename):  # Check if frame already exists
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if ret:
                cv2.imwrite(frame_filename, frame)
    cap.release()
    
# Check if there are already number_of_frames frames saved for both cameras
existing_frames_north = [f for f in os.listdir(output_dir) if f.startswith(f'{camera_b}_frame_')]
existing_frames_top = [f for f in os.listdir(output_dir) if f.startswith(f'{camera_a}_frame_')]

if len(existing_frames_north) >= number_of_frames and len(existing_frames_top) >= number_of_frames:
    print("Frames already exist for both cameras. No new frames will be saved.")
else:
    # Get total number of frames in the videos
    total_frames_a = get_total_frames(video_path_a)
    total_frames_b = get_total_frames(video_path_b)
    print(total_frames_b)

    # Ensure both videos have the same number of frames
    assert total_frames_a == total_frames_b, "Videos do not have the same number of frames."

    # Randomly select number_of_frames frame indices
    random_frame_indices = random.sample(range(total_frames_a), number_of_frames)
    print(random_frame_indices)

    # Extract and save frames from both videos
    extract_and_save_frames(video_path_a, random_frame_indices, f'{camera_a}')
    extract_and_save_frames(video_path_b, random_frame_indices, f'{camera_b}')

    print(f"Extracted frames saved to {output_dir}")

# 2. Image registration

In [ ]:
def load_images(camera_a_frame_path, camera_b_frame_path):
    """
    Load images from the given paths.
    """
    img_a = cv2.imread(camera_a_frame_path, cv2.IMREAD_GRAYSCALE)
    img_b = cv2.imread(camera_b_frame_path, cv2.IMREAD_GRAYSCALE)
    return img_a, img_b

def detect_and_match_features(img_a, img_b, nfeatures=1000):
    """
    Detect and match features between two images using ORB.
    """
    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=nfeatures)

    # Detect keypoints and descriptors
    kp_a, des_a = orb.detectAndCompute(img_a, None)
    kp_b, des_b = orb.detectAndCompute(img_b, None)

    # Match features using the Brute-Force matcher with Hamming distance
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des_a, des_b)
    matches = sorted(matches, key=lambda x: x.distance)  # Sort matches by distance

    return kp_a, kp_b, matches

def compute_homography(kp_a, kp_b, matches):
    """
    Compute the homography matrix to map Image B to Image A.
    """
    # Extract matched keypoints
    src_pts = np.float32([kp_b[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_a[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute homography using RANSAC
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H, mask

def warp_image(img_b, H, img_a_shape):
    """
    Warp Image B to Image A's perspective using the homography matrix.
    """
    # Apply homography to warp Image B
    img_b_aligned = cv2.warpPerspective(img_b, H, (img_a_shape[1], img_a_shape[0]))
    return img_b_aligned

def plot_results(img_a, img_b, img_b_aligned, kp_a, kp_b, matches, mask):
    """
    Plot the original and aligned images, and matched features with each match in a different color.
    """
    # Generate different colors for each match line
    colors = [(np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255)) for _ in matches]

    # Draw matches
    img_matches = cv2.drawMatches(img_a, kp_a, img_b, kp_b, matches, None, 
                                  matchColor=None, singlePointColor=None, 
                                  matchesMask=mask.ravel().tolist(), flags=2)

    # Plot the results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_a, cmap='gray')
    plt.title("Camera A (Original)")

    plt.subplot(1, 3, 2)
    plt.imshow(img_b, cmap='gray')
    plt.title("Camera B (Original)")

    plt.subplot(1, 3, 3)
    plt.imshow(img_b_aligned, cmap='gray')
    plt.title("Camera B (Aligned to A)")

    plt.figure(figsize=(10, 5))
    plt.imshow(img_matches)
    plt.title('Feature Matches')
    plt.show()

def calculate_pixel_mapping(kp_a, kp_b, H, img_shape):
    """
    Calculate the pixel mapping from Image A to Image B and vice versa.
    """
    height, width = img_shape
    map_a_to_b = np.zeros((height, width, 2), dtype=np.float32)
    map_b_to_a = np.zeros((height, width, 2), dtype=np.float32)

    # Create pixel grid for Image A
    grid_x, grid_y = np.meshgrid(np.arange(width), np.arange(height))

    # Flatten and combine to create homogeneous coordinates
    points_a = np.stack([grid_x.ravel(), grid_y.ravel(), np.ones(grid_x.size)], axis=-1).T

    # Map pixels from A to B using the homography
    points_b = H @ points_a
    points_b /= points_b[2, :]  # Convert to Cartesian coordinates

    # Store the results in the pixel map
    map_a_to_b[grid_y, grid_x] = points_b[:2].T.reshape(height, width, 2)

    # Calculate the inverse homography for mapping from B to A
    H_inv = np.linalg.inv(H)
    points_b_inv = H_inv @ points_b
    points_b_inv /= points_b_inv[2, :]  # Convert to Cartesian coordinates

    # Store the results in the pixel map
    map_b_to_a[grid_y, grid_x] = points_b_inv[:2].T.reshape(height, width, 2)

    return map_a_to_b, map_b_to_a

def multi_frame_registration(frame_paths_a, frame_paths_b):
    """
    Main function to perform registration using multiple frames to compute a more accurate homography.
    """
    homographies = []
    for frame_a_path, frame_b_path in zip(frame_paths_a, frame_paths_b):
        # Load Images
        img_a, img_b = load_images(frame_a_path, frame_b_path)
        
        # Detect and Match Features
        kp_a, kp_b, matches = detect_and_match_features(img_a, img_b)

        # Compute Homography
        H, mask = compute_homography(kp_a, kp_b, matches)
        if H is not None:
            homographies.append(H)
    
    # Average Homography
    H_avg = np.mean(homographies, axis=0)
    print(f"Averaged Homography:\n{H_avg}")

    # Use the first image shape for warping
    img_a, img_b = load_images(frame_paths_a[0], frame_paths_b[0])
    img_b_aligned = warp_image(img_b, H_avg, img_a.shape)
    
    # Plot Results
    plot_results(img_a, img_b, img_b_aligned, kp_a, kp_b, matches, mask)
    
    # Calculate Pixel Maps
    map_a_to_b, map_b_to_a = calculate_pixel_mapping(kp_a, kp_b, H_avg, img_a.shape)

    return H_avg, map_a_to_b, map_b_to_a

In [ ]:
# Collect frame paths
frame_paths_a = [os.path.join(output_dir, f'{camera_a}_frame_{i}.png') for i in range(number_of_frames)]
frame_paths_b = [os.path.join(output_dir, f'{camera_b}_frame_{i}.png') for i in range(number_of_frames)]

# Run the multi-frame registration and plot for first frame
H_avg, map_a_to_b, map_b_to_a = multi_frame_registration(frame_paths_a, frame_paths_b)
H_avg, map_a_to_b, map_b_to_a